# Проект по SQL. Анализ компьютерной игры

## Описание проблемы:

Разработчики утверждают, что игру можно пройти полностью без использования валюты «райские лепестки» и это опциональный способ ускорить прохождение. Однако для компании разработчика важно привлекать платящих игроков — тех, кто готов покупать внутриигровую валюту за реальные деньги. 

Задачи:
- выяснить, какая доля игроков покупает внутриигровую валюту «райские лепестки» за реальные деньги и есть ли зависимость доли платящих игроков от расы персонажа;
- детально изучить, как происходит покупка эпических предметов внутри игры. 

Цель проекта: изучить влияние характеристик игроков и их игровых персонажей на покупку внутриигровой валюты «райские лепестки», а также оценить активность игроков при совершении внутриигровых покупок

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-fantasy'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


### 1.1. Доля платящих пользователей по всем данным:

In [4]:
query = """
SELECT
    COUNT(*),
    COUNT(CASE WHEN payer = 1 THEN 1 ELSE NULL END),
    ROUND(
        COUNT(CASE WHEN payer = 1 THEN 1 ELSE NULL END) * 100.0 / NULLIF(COUNT(*), 0), 
        2
    ) 
FROM fantasy.users
"""
result = pd.read_sql(query, engine)
result

,count,count,round
0,22214,3929,17.69


### 1.2. Доля платящих пользователей в разрезе расы персонажа:

In [8]:
query = """
SELECT
    r.race,
    SUM(CASE WHEN u.payer = 1 THEN 1 ELSE 0 END) AS paying_users_count,
    COUNT(u.id) AS total_users_count,
    ROUND(SUM(CASE WHEN u.payer = 1 THEN 1 ELSE 0 END) * 100.0 / NULLIF(COUNT(u.id), 0), 2) AS paying_percentage
FROM
    fantasy.users u
JOIN
    fantasy.race r ON u.race_id = r.race_id
GROUP BY
    r.race
ORDER BY
    paying_percentage DESC;
"""

result = pd.read_sql(query, engine)
result

,race,paying_users_count,total_users_count,paying_percentage
0,Demon,238,1229,19.37
1,Hobbit,659,3648,18.06
2,Human,1114,6328,17.60
3,Orc,636,3619,17.57
4,Northman,626,3562,17.57
5,Angel,229,1327,17.26
6,Elf,427,2501,17.07


Доля платящих персонажей для всей игры (с общим кол-вом игроков 22.214) равна
примерно 17.69 %.
Мы можем заметить, что у расы Demon доля платящих самая наивысшая и равна
19.37%,далее идет Hobbit со значениями 18.06%, у всех остальных рас примерно
равняется доле для всей игры.

### Задача 2. Исследование внутриигровых покупок
### 2.1. Статистические показатели по полю amount:

In [9]:
query = """
SELECT 
    COUNT(*) AS total_purchases,
    SUM(amount) AS total_amount,
    MIN(amount) AS min_amount,
    MAX(amount) AS max_amount,
    ROUND(AVG(amount::numeric), 2) AS avg_amount,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY amount) AS median_amount,
    ROUND(STDDEV(amount::numeric), 2) AS amount_std_dev
FROM fantasy.events
"""
result = pd.read_sql(query, engine)
result

,total_purchases,total_amount,min_amount,max_amount,avg_amount,median_amount,amount_std_dev
0,1307678,686615040.0,0.0,486615.1,525.69,74.860001,2517.35


### 2.2: Аномальные нулевые покупки:

In [5]:
query = """
WITH purchase_stats AS (
    SELECT
        COUNT(*) AS total_purchases,
        COUNT(CASE WHEN amount = 0 THEN 1 END) AS zero_amount_purchases
    FROM fantasy.events
)
SELECT
    zero_amount_purchases AS zero_cost_purchases_count,
    total_purchases AS total_purchases_count,
    ROUND(zero_amount_purchases * 100.0 / NULLIF(total_purchases, 0), 2) AS zero_purchases_percentage
FROM purchase_stats
"""
result = pd.read_sql(query, engine)
result

,zero_cost_purchases_count,total_purchases_count,zero_purchases_percentage
0,907,1307678,0.07


### 2.3: Сравнительный анализ активности платящих и неплатящих игроков:

In [6]:
query = """
WITH player_activity AS (
    SELECT 
        u.id,
        u.payer,
        COUNT(e.amount) AS purchase_count,
        SUM(e.amount) AS total_spent
    FROM 
        fantasy.users u
    LEFT JOIN 
        fantasy.events e ON u.id = e.id
    WHERE e.amount > 0
    GROUP BY 
        u.id, u.payer
)
SELECT
    CASE 
        WHEN payer = 1 THEN 'Paying players'
        ELSE 'Non-paying players'
    END AS player_group,
    COUNT(id) AS total_players,
    ROUND(AVG(purchase_count::numeric), 2) AS avg_purchases,
    ROUND(AVG(total_spent::numeric), 2) AS avg_spent
FROM 
    player_activity
GROUP BY 
    player_group
"""
result = pd.read_sql(query, engine)
result

,player_group,total_players,avg_purchases,avg_spent
0,Non-paying players,11348,97.56,48631.72
1,Paying players,2444,81.68,55467.75


Можем заметить, что платящие игроки в среднем делают меньшее количество
покупок(81.68),чем неплатящие(97.56), однако тратят платящие
больше(55467.75),чем неплатящие(48631.74)

### 2.4: Популярные эпические предметы:

In [11]:
query = """
WITH 
total_stats AS (
    SELECT
        COUNT(*) AS total_sales,
        COUNT(DISTINCT id) AS total_buyers
    FROM fantasy.events
    WHERE amount > 0
),
item_stats AS (
    SELECT
        i.item_code,
        i.game_items AS item_name,
        COUNT(*) AS sales_count,
        COUNT(DISTINCT e.id) AS unique_buyers
    FROM
        fantasy.events e
    JOIN
        fantasy.items i ON e.item_code = i.item_code
    WHERE
        e.amount > 0
    GROUP BY
        i.item_code, i.game_items
)
SELECT
    item_name AS item_name,
    sales_count AS sales_count,
    ROUND(sales_count * 100.0 / NULLIF((SELECT total_sales FROM total_stats), 0), 2) AS sales_percentage,
    unique_buyers AS unique_buyers,
    ROUND(unique_buyers * 100.0 / NULLIF((SELECT total_buyers FROM total_stats), 0), 2) AS buyers_percentage
FROM
    item_stats
ORDER BY sales_count DESC
limit 7
"""
result = pd.read_sql(query, engine)
result

,item_name,sales_count,sales_percentage,unique_buyers,buyers_percentage
0,Book of Legends,1004516,76.87,12194,88.41
1,Bag of Holding,271875,20.81,11968,86.77
2,Necklace of Wisdom,13828,1.06,1627,11.80
3,Gems of Insight,3833,0.29,926,6.71
4,Treasure Map,3084,0.24,753,5.46
5,Amulet of Protection,1078,0.08,445,3.23
6,Silver Flask,795,0.06,633,4.59


Среди эпических предметов самыми популярными по продажам выделяются первые
три(у всех остальных доля продаж ниже 1%):

- Book of Legends - кол-во продаж - 1.004.516, доля от продаж - 76.87%, доля
покупателей - 88.41%

- Bag of Holding - кол-во продаж - 271.875, доля от продаж - 20.81%, доля
покупателей - 86.77%

- Necklace of Wisdom - кол-во продаж - 13.828, доля от продаж - 1.06%, доля
покупателей - 11.80%

### Часть 2. Решение ad hoc-задач
### Задача 1. Зависимость активности игроков от расы персонажа:

In [10]:
query = """
WITH 
race_population AS (
    SELECT 
        r.race,
        COUNT(DISTINCT u.id) AS total_players
    FROM 
        fantasy.users u
    JOIN 
        fantasy.race r ON u.race_id = r.race_id
    GROUP BY 
        r.race
),
buying_activity AS (
    SELECT 
        r.race,
        COUNT(DISTINCT u.id) AS players_with_purchases,
        COUNT(DISTINCT CASE WHEN u.payer = 1 THEN u.id END) AS paying_players
    FROM 
        fantasy.users u
    JOIN 
        fantasy.race r ON u.race_id = r.race_id
    JOIN 
        fantasy.events e ON u.id = e.id  
    WHERE 
        e.amount > 0
    GROUP BY 
        r.race
),
purchase_metrics AS (
    SELECT 
        r.race,
        COUNT(e.transaction_id) AS total_purchases,
        SUM(e.amount) AS total_amount_spent
    FROM 
        fantasy.events e
    JOIN 
        fantasy.users u ON e.id = u.id  
    JOIN 
        fantasy.race r ON u.race_id = r.race_id
    WHERE 
        e.amount > 0
    GROUP BY 
        r.race
)
SELECT
    rp.race AS race_name,
    rp.total_players AS total_players_count,
    COALESCE(ba.players_with_purchases, 0) AS buyers_count,
    ROUND(COALESCE(ba.players_with_purchases::numeric, 0) * 100.0 / NULLIF(rp.total_players::numeric, 0), 2) AS buyers_percentage,
    COALESCE(ba.paying_players, 0) AS paying_players_count,
    ROUND(COALESCE(ba.paying_players::numeric, 0) * 100.0 / 
          NULLIF(COALESCE(ba.players_with_purchases::numeric, 0), 0), 2) AS paying_ratio,
    ROUND(COALESCE(pm.total_purchases::numeric, 0) * 1.0 / 
          NULLIF(COALESCE(ba.players_with_purchases::numeric, 0), 0), 2) AS avg_purchases_per_player,
    ROUND(COALESCE(pm.total_amount_spent::numeric, 0) * 1.0 / 
          NULLIF(COALESCE(pm.total_purchases::numeric, 0), 0), 2) AS avg_purchase_amount,
    ROUND(COALESCE(pm.total_amount_spent::numeric, 0) * 1.0 / 
          NULLIF(COALESCE(ba.players_with_purchases::numeric, 0), 0), 2) AS avg_spending_per_player
FROM
    race_population rp
LEFT JOIN
    buying_activity ba ON rp.race = ba.race
LEFT JOIN
    purchase_metrics pm ON rp.race = pm.race
ORDER BY
    rp.race
"""
result = pd.read_sql(query, engine)
result

,race_name,total_players_count,buyers_count,buyers_percentage,paying_players_count,paying_ratio,avg_purchases_per_player,avg_purchase_amount,avg_spending_per_player
0,Angel,1327,820,61.79,137,16.71,106.80,455.68,48668.54
1,Demon,1229,737,59.97,147,19.95,77.87,529.05,41197.29
2,Elf,2501,1543,61.70,251,16.27,78.79,682.30,53758.72
3,Hobbit,3648,2266,62.12,401,17.70,86.13,552.87,47617.83
4,Human,6328,3921,61.96,706,18.01,121.40,403.12,48939.56
5,Northman,3562,2229,62.58,406,18.21,82.10,761.47,62518.17
6,Orc,3619,2276,62.89,396,17.40,81.74,510.87,41757.64


В игре есть всего 7 рас. Общее кол-во игроков,которые выбрали определенную расу
варьируется от 1.229(Demon) до 6.328(Human). Однако несмотря на разницу, доля
покупателей от общего числа пользователей у всех рас практически одинаковая(от
59,97% до 62,12%). Доля платящих среди покупателей также практически не
отличается (от 17,4% до 19,95%).
Больше всего покупок в среднем делают по расе Human(121,4), у всех остальных рас
от 77,87 до 121,4. Однако средняя стоимость 1 покупки выше всего у расы Elf
(682.33), при том, что во всех остальных случаях стоимость от 403,07 до 682,33 -
не сильно отличается.
По средним затратам на игрока превышает всего раса Northman(62.519,07), однако
у всех остальных они составляют от 41.194,44 до 62.519,07
Вывод: Прохождение игры за персонажей разных рас требует примерно равного
количества покупок эпических предметов

## Рекомендации:

Для повышения монетизации игры рекомендуем следующие меры: внедрить эксклюзивные акции для рас Demon и Hobbit, где доля платящих игроков максимальна; оптимизировать ценовую политику, добавив промежуточные ценовые сегменты между 75 и 500 единицами; для топовых предметов (Book of Legends, Bag of Holding) разработать специальные предложения и лимитированные версии, а малопопулярные предметы перебалансировать; создать уникальный контент для каждой расы с учетом их особенностей - накопительные бонусы для Human и премиум-контент для Elf; провести аудит системы покупок для выявления причин нулевых транзакций; внедрить систему подписки с бонусами для платящих игроков и пробные версии платного контента со скидками для остальных пользователей. 